In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np
import math
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()

In [3]:
video_directory = r"C:\Users\eissa\dd\videos"
videos_with_labels = {}
for filename in os.listdir(video_directory):
    if filename.endswith(".mp4"):
        patient_label = filename.replace(".mp4", "")
        full_path = os.path.join(video_directory, filename)
        videos_with_labels[full_path] = patient_label

fieldnames = ["patient_label", "hip_knee_angle", "knee_ankle_angle", "hip_knee_angle_r", "knee_ankle_angle_r",
              "left_shoulder_elbow_angle", "left_elbow_wrist_angle", "right_shoulder_elbow_angle", "right_elbow_wrist_angle"]

In [4]:
detailed_data = []
summary_data = {}

for video_path, patient_label in videos_with_labels.items():
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        continue

    if patient_label not in summary_data:
        summary_data[patient_label] = {field: [] for field in fieldnames[1:]}  # Skip 'patient_label' field

    detailed_data.append({"patient_label": patient_label})

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break

        h, w, c = img.shape
        new_w, new_h = 600, int(h * (600 / w))
        img = cv2.resize(img, (new_w, new_h))

        results = pose.process(img)

        opImg = np.zeros([new_h, new_w, c], dtype=np.uint8)
        opImg.fill(50)

                # Draw landmarks
        mp_draw.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                               mp_draw.DrawingSpec((255, 0, 0), 2, 2),
                               mp_draw.DrawingSpec((255, 0, 255), 2, 2)
                               )
        # extract landmarks
        if results.pose_landmarks is not None:

            landmarks = results.pose_landmarks.landmark
            hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            hip_r = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            knee_r = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            ankle_r = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]



            # calculate the angles
            hip_knee_angle = math.degrees(math.atan2(knee.y - hip.y, knee.x - hip.x))
            knee_ankle_angle = math.degrees(math.atan2(ankle.y - knee.y, ankle.x - knee.x))
            hip_knee_angle_r = math.degrees(math.atan2(knee_r.y - hip_r.y, knee_r.x - hip_r.x))
            knee_ankle_angle_r = math.degrees(math.atan2(ankle_r.y - knee_r.y, ankle_r.x - knee_r.x))
            left_shoulder_elbow_angle = math.degrees(math.atan2(left_elbow.y - left_shoulder.y, left_elbow.x - left_shoulder.x))
            left_elbow_wrist_angle = math.degrees(math.atan2(left_wrist.y - left_elbow.y, left_wrist.x - left_elbow.x))
            right_shoulder_elbow_angle = math.degrees(math.atan2(right_elbow.y - right_shoulder.y, right_elbow.x - right_shoulder.x))
            right_elbow_wrist_angle = math.degrees(math.atan2(right_wrist.y - right_elbow.y, right_wrist.x - right_elbow.x))



            # draw lines on the body parts
            cv2.line(opImg, (int(hip.x * w), int(hip.y * h)), (int(knee.x * w), int(knee.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(knee.x * w), int(knee.y * h)), (int(ankle.x * w), int(ankle.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(hip_r.x * w), int(hip_r.y * h)), (int(knee_r.x * w), int(knee_r.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(knee_r.x * w), int(knee_r.y * h)), (int(ankle_r.x * w), int(ankle_r.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(left_shoulder.x * w), int(left_shoulder.y * h)), (int(left_elbow.x * w), int(left_elbow.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(left_elbow.x * w), int(left_elbow.y * h)), (int(left_wrist.x * w), int(left_wrist.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(right_shoulder.x * w), int(right_shoulder.y * h)), (int(right_elbow.x * w), int(right_elbow.y * h)), (0, 255, 0), 3)
            cv2.line(opImg, (int(right_elbow.x * w), int(right_elbow.y * h)), (int(right_wrist.x * w), int(right_wrist.y * h)), (0, 255, 0), 3)


            # display the angles as text next to the lines
            cv2.putText(opImg, "{:.2f}".format(hip_knee_angle), (int(knee.x * w) + 10, int(knee.y * h) + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(knee_ankle_angle), (int(ankle.x * w) + 10, int(ankle.y * h) + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(hip_knee_angle_r), (int(knee_r.x * w) + 10, int(knee_r.y * h) + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(knee_ankle_angle_r), (int(ankle_r.x * w) + 10, int(ankle_r.y * h) + 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(left_shoulder_elbow_angle), (int(left_elbow.x * w) + 10, int(left_elbow.y * h) + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(left_elbow_wrist_angle), (int(left_wrist.x * w) + 10, int(left_wrist.y * h) + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(right_shoulder_elbow_angle), (int(right_elbow.x * w) + 10, int(right_elbow.y * h) + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(opImg, "{:.2f}".format(right_elbow_wrist_angle), (int(right_wrist.x * w) + 10, int(right_wrist.y * h) + 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
             # Append detailed and summary data
                
            frame_data = {
                "hip_knee_angle": hip_knee_angle,
                "knee_ankle_angle": knee_ankle_angle,
                "hip_knee_angle_r": hip_knee_angle_r,
                "knee_ankle_angle_r": knee_ankle_angle_r,
                "left_shoulder_elbow_angle": left_shoulder_elbow_angle,
                "left_elbow_wrist_angle": left_elbow_wrist_angle,
                "right_shoulder_elbow_angle": right_shoulder_elbow_angle,
                "right_elbow_wrist_angle": right_elbow_wrist_angle,
            }


            detailed_data.append(frame_data)
            for field in summary_data[patient_label]:
                summary_data[patient_label][field].append(frame_data[field])

               

        cv2.imshow("Pose Estimation", img)
        cv2.imshow("Extracted Pose", opImg)

        if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
cv2.destroyAllWindows()

In [6]:
fieldnames = ["patient_label", "hip_knee_angle", "knee_ankle_angle", "hip_knee_angle_r", "knee_ankle_angle_r",
              "left_shoulder_elbow_angle", "left_elbow_wrist_angle", "right_shoulder_elbow_angle", "right_elbow_wrist_angle"]

with open("detailed_angles.csv", mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for row in detailed_data:
        writer.writerow(row)


In [7]:
# Writing summary data (average values) to another CSV
with open("summary_angles.csv", mode='w', newline='') as file:
    fieldnames_summary = ["patient_label"] + ["avg_" + field for field in fieldnames[1:]]
    writer = csv.DictWriter(file, fieldnames=fieldnames_summary)
    writer.writeheader()
    for label, data in summary_data.items():
        avg_data = {f'avg_{field}': np.mean(vals) for field, vals in data.items() if vals}
        writer.writerow({"patient_label": label, **avg_data})

In [8]:
df = pd.read_csv(r"C:\Users\eissa\dd\detailed_angles.csv")
df

,patient_label,hip_knee_angle,knee_ankle_angle,hip_knee_angle_r,knee_ankle_angle_r,left_shoulder_elbow_angle,left_elbow_wrist_angle,right_shoulder_elbow_angle,right_elbow_wrist_angle
0,OAW01-bottom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,100.617182,89.490668,79.066984,103.316816,84.065700,78.285235,112.684778,102.451913
2,NaN,101.571657,91.298998,78.928803,110.164716,80.677259,82.082049,112.253828,102.499115
3,NaN,101.903664,94.777712,77.878505,116.895911,79.217586,85.281872,111.248798,102.271423
4,NaN,102.051940,97.171604,78.653020,115.105438,78.933753,86.116666,109.465157,101.082634
...,...,...,...,...,...,...,...,...,...
42500,NaN,97.146558,117.001796,92.623096,89.220247,114.036943,113.871904,79.812862,81.267861
42501,NaN,96.155070,112.208445,92.626370,89.132014,109.029911,114.175233,81.318354,83.537148
42502,NaN,95.004491,109.549543,92.033644,88.843111,109.724468,112.136585,82.609815,82.856479
42503,NaN,94.658698,107.089324,91.411576,87.123394,109.593082,108.794992,78.188324,83.459491


In [ ]:
# Open the CSV file
with open('angles.csv', 'r') as infile:
    reader = csv.reader(infile)

    # Create a list of even rows
    even_rows = [row for i, row in enumerate(reader) if i % 2 == 1]

# Write the even rows to a new CSV file
with open('output_file.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(even_rows)
    
df = pd.read_csv('output_file.csv')



In [ ]:

header = ["hip_knee_angle", "knee_ankle_angle", "hip_knee_angle_r", "knee_ankle_angle_r", "left_shoulder_elbow_angle", "left_elbow_wrist_angle", "right_shoulder_elbow_angle", "right_elbow_wrist_angle"]
len(header)

In [ ]:
df.to_csv("angles_updated.csv", header=header, index=False)

In [ ]:
df_2 = pd.read_csv('angles_updated.csv')
df_2

In [ ]:
print(df_2.columns)


In [ ]:
y_data = df_2['knee_ankle_angle'].values
print(y_data)

In [ ]:
y_data = df_2['knee_ankle_angle'].values
x_data = range(0, len(y_data))  # Use the length of y_data as the x-axis values
fig, ax = plt.subplots()
ax.plot(x_data, y_data)
ax.set_xlabel('Index')
ax.set_ylabel('Hip Knee Angle')
ax.set_ylim([0, 180])
plt.show()

In [ ]:
df_3 = pd.read_csv(r'C:\Users\eissa\dd\patinet_data_angle_ graph.csv')

In [ ]:
y_data = df_3['hip_knee_angle'].values
x_data = range(0, len(y_data))  # Use the length of y_data as the x-axis values
fig, ax = plt.subplots()
ax.plot(x_data, y_data)
ax.set_xlabel('Index')
ax.set_ylabel('Hip Knee Angle')
ax.set_ylim([0, 180])
plt.show()